In [1]:
import pandas as pd
import spacy
from tqdm import tqdm

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx
import json

import matplotlib.pyplot as plt

import timeit

2023-03-14 23:56:34.488654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 23:56:35.197915: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-14 23:56:35.197937: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-14 23:56:37.399601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Análise dos dados news.tsv
- Quantidade de linhas com entities de title e abstract nulas

In [2]:
newsFile = "dataset/news.tsv"
newsDf = pd.read_csv(newsFile, header=None, sep="\t", skiprows = 0, names=[
                  'id', 'category', 'subcategory', 'title', 'abstract', 'url',
                  'title_entities', 'abstract_entities'
              ]) #, nrows=500
newsDf = newsDf.fillna("")
newsDf['title_entities'] = newsDf[['title_entities']].apply(lambda x: x.str.strip()).replace('', '[]')
newsDf['abstract_entities'] = newsDf[['abstract_entities']].apply(lambda x: x.str.strip()).replace('', '[]')

In [3]:
newsDf

,id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
...,...,...,...,...,...,...,...,...
101522,N115249,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
101523,N64337,finance,finance-real-estate,Mansion Monday: Contemporary Des Moines home i...,Among the perks of this unique Des Moines home...,https://assets.msn.com/labs/mind/BBWzQq8.html,"[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W...","[{""Label"": ""Des Moines, Iowa"", ""Type"": ""G"", ""W..."
101524,N100102,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]
101525,N74617,autos,autossports,Best Sports Car Deals for October,,https://assets.msn.com/labs/mind/BBy5rVe.html,"[{""Label"": ""Peugeot RCZ"", ""Type"": ""V"", ""Wikida...",[]


## Processamento
- Processar entidades
    - Cada entidade com Label, Type, WikiDataId
    - Concatenar entidades de title e abstract, remover duplicatas
- Iterar notícias:
    - Criar nodo da notícia apenas se existir pelo menos 1 entidade
    - Adicionar aresta "related" entre todas os nodos do tipo entidade de uma mesma notícia
    - Adicionar aresta "content" entre a notícia e cada um de seus nodos do tipo entidade 
- Iterar usuários:
    - Criar nodo do usuário
    - Adicionar aresta "reading" entre nodo do usuário e cada um dos nodos das notícias que ele leu (se existir)
    
    
- Sistema relação entre objetos: se já existir uma relação entre 2 entidades, somar um contador na aresta
- Cada nodo com nome e tipo (notícia, usuário ou entity)
    - Notícia com campo data e flag "new"


In [2]:
from neo4j import GraphDatabase


# WRITE

@staticmethod
def createEntity(tx, typee, label, WDId):
    result = tx.run("MERGE (a:Entity{label: $label, typee: $typee, WDId: $WDId}) ", label=label, typee=typee, WDId=WDId)
    return

@staticmethod
def createNews(tx, ID, category, subcategory, title, abstract, url, new):
    result = tx.run("MERGE (a:News{id: $ID, title: $title, abstract: $abstract, category: $category, subcategory: $subcategory, url: $url, new: $new}) ", ID=ID, title=title, abstract=abstract, category=category, subcategory=subcategory, url=url, new=new)
    return

@staticmethod
def createUser(tx, ID, new): #new/last acess
    result = tx.run("MERGE (a:User{id: $ID, new:$new}) ", ID=ID, new=new)
    return

@staticmethod
def connectUserToNews(tx, userID, newsID, time):
    result = tx.run("MATCH (u:User), (n:News) "
                    "WHERE u.id=$userID AND n.id=$newsID "
                    "MERGE (u)-[rd1:READ]->(n) "
                    "MERGE (n)-[rd2:READ]->(u) "
                    "SET rd1.time=$time "
                    "SET rd2.time=$time ", userID=userID, newsID=newsID, time=time)
    return

#NEW
@staticmethod
def createUserAndConnectUserToNews(tx, userID, new, newsID, time):
    result = tx.run("MERGE (a:User{id: $userID, new:$new}) WITH 1 as dummy "
                    "MATCH (u:User), (n:News) "
                    "WHERE u.id=$userID AND n.id=$newsID "
                    "MERGE (u)-[rd1:READ]->(n) "
                    "MERGE (n)-[rd2:READ]->(u) "
                    "SET rd1.time=$time "
                    "SET rd2.time=$time ", userID=userID, new=new, newsID=newsID, time=time)
    return

@staticmethod
def connectEntityToEntity(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity), (e2:Entity) "
                    "WHERE e1.label=$entity1Label AND e2.label=$entity2Label "
                    "MERGE (e1)-[rel1:RELATED]->(e2) "
                    "MERGE (e2)-[rel2:RELATED]->(e1) "
                    "SET rel1.nCon=1 "
                    "SET rel2.nCon=1 ", entity1Label=entity1Label, entity2Label=entity2Label)
    return

@staticmethod
def connectNewsToEntity(tx, newsID, entityLabel):
    result = tx.run("MATCH (n:News), (e:Entity) "
                    "WHERE n.id=$newsID AND e.label=$entityLabel "
                    "MERGE (n)-[con1:CONTENT]->(e) "
                    "MERGE (e)-[con2:CONTENT]->(n) ", newsID=newsID, entityLabel=entityLabel)
    return


@staticmethod
def updateNConEntToEnt(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity {label: $entity1Label}), (e2:Entity {label: $entity2Label}) " 
                    "MATCH (e1)-[rel1:RELATED]->(e2) "
                    "MATCH (e2)-[rel2:RELATED]->(e1) "
                    "SET rel1.nCon=rel1.nCon+1 "
                    "SET rel2.nCon=rel2.nCon+1 ", entity1Label=entity1Label, entity2Label=entity2Label)
    return 

# READ

@staticmethod
def entToEntConExists(tx, entity1Label, entity2Label):
    result = tx.run("MATCH (e1:Entity {label: $entity1Label}), (e2:Entity {label: $entity2Label}) " 
                    "MATCH (e1)-[rel:RELATED]->(e2) "
                    "WITH COUNT(rel) > 0  as node_exists "
                    "RETURN node_exists ", entity1Label=entity1Label, entity2Label=entity2Label)
    return result.fetch(1)[0][0]


@staticmethod
def newsExists(tx, newsID):
    result = tx.run("MATCH (n:News {id: $newsID}) " 
                    "WITH COUNT(n) > 0  as node_exists "
                    "RETURN node_exists ", newsID=newsID)
    return result.fetch(1)[0][0]



In [3]:
uri="bolt://localhost:7687"
user="neo4j"
password="88888888"

driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
%%time
with driver.session() as session:
    for index, row in newsDf.iterrows():

        titleEntities = json.loads(row["title_entities"])
        abstractEntities = json.loads(row["abstract_entities"])
        entitiesLabel = []
        entities = []
        for ent in titleEntities:
            if ent['Label'] not in entitiesLabel:
                entities.append(ent)
                entitiesLabel.append(ent['Label'])
        for ent in abstractEntities:
            if ent['Label'] not in entitiesLabel:
                entities.append(ent)
                entitiesLabel.append(ent['Label'])

        if(entitiesLabel == []):
            continue
        
        # Insert News
        #print(row['id'] + " ---> " + row['title'])
        session.execute_write(createNews, row['id'], row['category'], row['subcategory'], row['title'], row['abstract'], row['url'], '0')
        
        # Insert Entities
        for ent in entities:
            session.execute_write(createEntity, ent['Type'], ent['Label'], ent['WikidataId'])
            session.execute_write(connectNewsToEntity, row['id'], ent['Label'])
        
        count = 1
        for ent1 in entities:
            for ent2 in entities[count:]:
                if ent1['Label'] != ent2['Label']:
                    exists = session.execute_read(entToEntConExists, ent1['Label'], ent2['Label'])
                    if(exists):
                        session.execute_write(updateNConEntToEnt, ent1['Label'], ent2['Label'])
                    else:
                        session.execute_write(connectEntityToEntity, ent1['Label'], ent2['Label'])
            count+=1


## Análise dos dados behaviors.tsv

In [10]:
behavFile = "dataset/behaviors.tsv"
behavDf = pd.read_csv(behavFile, header=None, sep="\t", skiprows = 0, nrows=1400000, names=[
                  'impression_id', 'user_id', 'time', 'history', 'impressions'
              ]) #, nrows=50000
behavDf = behavDf.fillna("")

In [6]:
behavDf

,impression_id,user_id,time,history,impressions
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...
...,...,...,...,...,...
999995,999996,U289384,11/12/2019 8:06:53 PM,N102357 N11663 N73933 N75484 N33286 N33286 N11266,N37930-0 N10252-0 N63235-0 N13761-0 N98861-1 N...
999996,999997,U705172,11/12/2019 5:13:32 PM,N72187 N72174 N12959 N65391 N58992 N47522 N102...,N10012-0 N94421-0 N106991-0 N55204-1 N54239-0 ...
999997,999998,U473020,11/14/2019 7:22:54 PM,N77559 N108217 N25671 N32176 N81220 N107151 N6...,N1767-0 N76665-0 N104990-0 N91238-0 N22627-0 N...
999998,999999,U186488,11/14/2019 4:38:44 AM,N67397 N102506 N9375 N42209 N51724 N108116 N94...,N58641-0 N85664-0 N54296-1 N102458-0 N55725-0 ...


In [4]:
len(behavDf)

NameError: name 'behavDf' is not defined

In [11]:
%%time
count = 0
totalLen = len(behavDf)
start = timeit.default_timer()
totalTime = 0
with driver.session() as session:    
    for index, row in behavDf.iterrows():
        #if count == 6:
        #    break
        if count%1000 == 0 and count >= 1141000:
            stop = timeit.default_timer()
            totalTime += (stop-start)
            print(str(count)+"/"+str(totalLen)+" Time: "+str(stop-start))
            if(count%5000 == 0):
                print("Total Time:" + str(totalTime))
            start = timeit.default_timer()
        count+=1
        if count < 1141000:
            continue
        newsList = []
            
        newsToParse = row['impressions'].split(" ")
        for news in newsToParse:
            splitedNewsFormat = news.split("-")
            newsID = splitedNewsFormat[0]
            newsClicked = splitedNewsFormat[1]
            if newsClicked == "1":
                newsList.append(newsID)

       
        if len(newsList)>0:
            #userCreated = False
            time = row['time']
            
            for newsID in newsList:
                session.execute_write(createUserAndConnectUserToNews, row['user_id'], '0', newsID, time)
                #exists = session.execute_read(newsExists, newsID)
                #print(newsID + ": " + str(exists))
                #if not exists:
                #    continue
                #if not userCreated:
                    #print(row['user_id'])
                    #session.execute_write(createUser, row['user_id'], '0')
                    #userCreated = True
                #session.execute_write(connectUserToNews, row['user_id'], newsID, time)
                

1141000/1400000 Time: 59.31371069594752
1142000/1400000 Time: 675.1091575480532
1143000/1400000 Time: 671.9955905909883
1144000/1400000 Time: 703.9608750829939
1145000/1400000 Time: 690.0378303879406
Total Time:2800.4171643059235
1146000/1400000 Time: 672.3643818059936
1147000/1400000 Time: 622.8884457109962
1148000/1400000 Time: 661.9133980500046
1149000/1400000 Time: 676.5298306669574
1150000/1400000 Time: 686.0147267429857
Total Time:6120.127947282861
1151000/1400000 Time: 678.6894125689287
1152000/1400000 Time: 664.53160359303
1153000/1400000 Time: 658.2874975679442
1154000/1400000 Time: 654.8435763230082
1155000/1400000 Time: 671.5466528150719
Total Time:9448.026690150844
1156000/1400000 Time: 739.3596593269613
1157000/1400000 Time: 699.4294356770115
1158000/1400000 Time: 693.972709517926
1159000/1400000 Time: 719.663104754989
1160000/1400000 Time: 654.3482572759967
Total Time:12954.799856703728
1161000/1400000 Time: 718.1972332500154
1162000/1400000 Time: 702.5952293779701
116300

KeyboardInterrupt: 

In [29]:
driver.close()